In [1]:
import sys
sys.path.append('../')

import networkx as nx
from node2vec import Node2Vec

from src.utils import load_embeddings

%load_ext autoreload
%autoreload 2

In [2]:
G = nx.read_gpickle('data/network.gpickle')
G.number_of_nodes(), G.number_of_edges()

(60789, 75647)

To search for node embeddings I use node2vec approach. I tried to use network SVD from network-sklearn, but it didn't work well (I got very low scaled vectors with values about 1e-20). node2vec gives more pleasant embeddings in terms of using dot product or cosine similarity. By the way it's not so fast (model trains for 3,5 minutes for graph with 60k vertices and 75k edges with some hyperparameters tuning)

In [5]:
def generate_node_embeddings(graph: nx.Graph):
    model = Node2Vec(graph, dimensions=64, num_walks=10, walk_length=15, temp_folder='temp/', workers=3)
    result = model.fit(window=5, min_count=1)
    return result

In [6]:
%%time
embeddings = generate_node_embeddings(G)

Generating walks (CPU: 1): 100%|██████████| 4/4 [07:51<00:00, 117.86s/it]


CPU times: user 10min 5s, sys: 2min 17s, total: 12min 22s
Wall time: 8min 43s


In [7]:
for idx, e in enumerate(G.edges()):
    if idx == 5:
        break
    print(e)

(0, 1)
(0, 24)
(0, 350192)
(1, 2)
(2, 3)


In [8]:
embeddings.wv.most_similar('0')

[('1', 0.9911059737205505),
 ('350192', 0.9812692999839783),
 ('24', 0.9794847965240479),
 ('2', 0.9694258570671082),
 ('3', 0.939180850982666),
 ('200921', 0.9313855171203613),
 ('350191', 0.9247511625289917),
 ('200923', 0.899285614490509),
 ('332388', 0.8979104161262512),
 ('211643', 0.8925637006759644)]

In [10]:
embeddings.save('models_data/embeddings64.data')

In [ ]:
# join embeddings with corresponding nodes (as x feature)
for idx, node in enumerate(G.nodes()):
    G.add_node(int(node), x=embeddings.wv[str(node)].copy())

In [ ]:
nx.readwrite.write_gpickle(G, 'data/network64.gpickle')